In [67]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [68]:
df = pd.read_csv('/content/gdrive/MyDrive/ML group Proj 2/MOVIE_reviews_from_rottentoms - Sheet1.csv')

In [69]:
df

,Review,Label
0,We thought it was a feel good movie. My grands...,positive
1,If you like original gut wrenching laughter yo...,positive
2,I saw this movie when I was about 12 when it c...,negative
3,"I remember this film,it was the first film i h...",positive
4,This was the worst movie I saw at WorldFest an...,negative
...,...,...
244,I loved this film. It was not what I was expec...,NaN
245,Don't Worry Darling Is Just A Harry Styles Mis...,negative
246,"Beautifully shot and acted, great audio too",positive
247,What a terrible movie! So so so boring,negative


In [70]:

df['Label'] = [1 if Label == 'positive' else 0 for Label in df['Label'].values]
df.head()

,Review,Label
0,We thought it was a feel good movie. My grands...,1
1,If you like original gut wrenching laughter yo...,1
2,I saw this movie when I was about 12 when it c...,0
3,"I remember this film,it was the first film i h...",1
4,This was the worst movie I saw at WorldFest an...,0


In [71]:
review = df['Review'].values
label = df['Label'].values

train_review = review[:180]
train_label = label[:180]


test_review = review[180:]
test_label = label[180:]
train_review.shape, test_review.shape

((180,), (69,))

In [72]:
from bs4 import BeautifulSoup
import numpy as np
import re
import tqdm
import unicodedata


def strip_html_tags(text):
  soup = BeautifulSoup(text, "html.parser")
  [s.extract() for s in soup(['iframe', 'script'])]
  stripped_text = soup.get_text()
  stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
  return stripped_text

def remove_accented_chars(text):
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  return text

def pre_process_corpus(docs):
  norm_docs = []
  for doc in tqdm.tqdm(docs):
    doc = strip_html_tags(doc)
    doc = doc.translate(doc.maketrans("\n\t\r", "   "))
    doc = doc.lower()
    doc = remove_accented_chars(doc)
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = re.sub(' +', ' ', doc)
    doc = doc.strip()  
    norm_docs.append(doc)
  
  return norm_docs

In [73]:
train_review = np.array(pre_process_corpus(train_review))
test_review = np.array(pre_process_corpus(test_review))

100%|██████████| 69/69 [00:00<00:00, 6762.16it/s]


In [74]:
train_review

array(['we thought it was a feel good movie my grandson said it was amazing',
       'if you like original gut wrenching laughter you will like this movie if you are young or old then you will love this movie hell even my mom liked it',
       'i saw this movie when i was about 12 when it came out i recall the scariest scene was the big bird eating men dangling helplessly from parachutes right out of the air the horror the horroras a young kid going to these cheesy b films on saturday afternoons i still was tired of the formula for these monster type movies that usually included the hero a beautiful woman who might be the daughter of a professor and a happy resolution when the monster died in the end i didnt care much for the romantic angle as a 12 year old and the predictable plots i love them now for the unintentional humorbut about a year or so later i saw psycho when it came out and i loved that the star janet leigh was bumped off early in the film i sat up and took notice at that 

In [75]:
import numpy as np
from numpy import array 
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D, Dropout
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

In [76]:
from sklearn.model_selection import train_test_split
token = Tokenizer(num_words=5000)
token.fit_on_texts(train_review)

In [77]:
token.index_word

{1: 'the',
 2: 'and',
 3: 'a',
 4: 'of',
 5: 'it',
 6: 'to',
 7: 'i',
 8: 'this',
 9: 'movie',
 10: 'is',
 11: 'was',
 12: 'in',
 13: 'that',
 14: 'for',
 15: 'but',
 16: 'with',
 17: 'you',
 18: 'film',
 19: 'as',
 20: 'so',
 21: 'not',
 22: 'on',
 23: 'its',
 24: 'one',
 25: 'all',
 26: 'like',
 27: 'have',
 28: 'just',
 29: 'about',
 30: 'be',
 31: 'at',
 32: 'from',
 33: 'good',
 34: 'are',
 35: 'some',
 36: 'his',
 37: 'see',
 38: 'my',
 39: 'if',
 40: 'an',
 41: 'bad',
 42: 'very',
 43: 'or',
 44: 'out',
 45: 'they',
 46: 'he',
 47: 'would',
 48: 'story',
 49: 'me',
 50: 'were',
 51: 'great',
 52: 'plot',
 53: 'much',
 54: 'had',
 55: 'by',
 56: 'more',
 57: 'time',
 58: 'movies',
 59: 'who',
 60: 'watch',
 61: 'even',
 62: 'up',
 63: 'there',
 64: 'what',
 65: 'no',
 66: 'how',
 67: 'we',
 68: 'love',
 69: 'seen',
 70: 'your',
 71: 'really',
 72: 'will',
 73: 'when',
 74: 'acting',
 75: 'life',
 76: 'can',
 77: 'made',
 78: 'has',
 79: 'only',
 80: 'well',
 81: 'dont',
 82: 'bec

In [78]:
encoded_train_reviews = token.texts_to_sequences(train_review)
#encoded_train_reviews = np.asarray(encoded_train_reviews)
encoded_test_reviews = token.texts_to_sequences(test_review)
#encoded_test_reviews = np.asarray(encoded_test_reviews)
encoded_test_reviews

[[28, 337, 1, 469, 16, 2, 111, 9, 4, 1, 182],
 [130, 2, 42],
 [1,
  91,
  12,
  1,
  9,
  519,
  44,
  6,
  30,
  1,
  91,
  1,
  1184,
  1171,
  519,
  44,
  6,
  30,
  3,
  8,
  18,
  2045,
  6,
  1,
  1171,
  12,
  821,
  131,
  21,
  127,
  70,
  57],
 [313, 620],
 [297,
  42,
  606,
  6,
  1007,
  16,
  2,
  7,
  11,
  20,
  543,
  16,
  1,
  98,
  13,
  7,
  342,
  5,
  148,
  81,
  2274,
  89],
 [8,
  10,
  3,
  42,
  2,
  1016,
  1946,
  9,
  88,
  167,
  7,
  7,
  1057,
  118,
  21,
  1576,
  149,
  1,
  1,
  1723,
  34,
  51,
  16,
  199,
  612,
  2,
  335,
  19,
  1319,
  59,
  2481,
  12,
  1,
  18,
  7,
  139,
  1,
  4,
  220,
  550,
  132,
  1,
  1470,
  76,
  131,
  3,
  152,
  16,
  3,
  135,
  8,
  10,
  321,
  37,
  9],
 [7,
  120,
  270,
  1,
  156,
  20,
  7,
  711,
  484,
  14,
  3,
  1251,
  43,
  752,
  126,
  162,
  127,
  4,
  38,
  57],
 [55,
  253,
  1,
  111,
  9,
  12,
  600,
  14,
  24,
  696,
  146,
  201,
  146,
  2,
  7,
  585,
  114,
  102,
  5,
  13,


In [79]:
vocab_size = len(token.word_index) + 1

maxlen = 100

X_train = pad_sequences(encoded_train_reviews , padding='post', maxlen=maxlen)
X_test = pad_sequences(encoded_test_reviews , padding='post', maxlen=maxlen)

X_train.shape

(180, 100)

In [80]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

glove_file = open('/content/gdrive/MyDrive/ML group Proj 2/Copy of glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

In [81]:
embedding_matrix = zeros((vocab_size, 100))
for word, index in token.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.038194  , -0.24487001,  0.72812003, ..., -0.1459    ,
         0.82779998,  0.27061999],
       [-0.071953  ,  0.23127   ,  0.023731  , ..., -0.71894997,
         0.86894   ,  0.19539   ],
       ...,
       [ 0.10535   , -0.10244   ,  0.34797001, ..., -0.55469   ,
        -0.28312001, -0.10072   ],
       [ 0.099456  ,  0.39107999,  0.27878001, ..., -0.51784003,
         0.72074997,  0.93941998],
       [ 0.058402  ,  0.03731   ,  0.36183   , ..., -0.56536001,
        -0.025931  ,  0.93856001]])

In [82]:
# since we have only 250 reviews we can use K-fold validation to build our model

k = 10
num_val_samples = len(X_train) // 10
num_epochs = 100
all_scores = []
for i in range(k):
    print(f"Processing fold #{i}")
    val_data = X_train[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_label[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate(
        [X_train[:i * num_val_samples],
         X_train[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_label[:i * num_val_samples],
         train_label[(i + 1) * num_val_samples:]],
        axis=0)

Processing fold #0
Processing fold #1
Processing fold #2
Processing fold #3
Processing fold #4
Processing fold #5
Processing fold #6
Processing fold #7
Processing fold #8
Processing fold #9


In [83]:
from keras.layers import Dense, Conv1D , MaxPooling1D, Flatten, Dropout
model = Sequential()
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)
model.add(Conv1D(64, 5, activation='relu'))
model.add(Conv1D(128,5, activation = 'relu'))
model.add(MaxPooling1D((2)))
model.add(Dropout(0.25))
model.add(Dense(64, activation = 'relu'))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [84]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 100, 100)          254900    
                                                                 
 conv1d_10 (Conv1D)          (None, 96, 64)            32064     
                                                                 
 conv1d_11 (Conv1D)          (None, 92, 128)           41088     
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 46, 128)          0         
 1D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 46, 128)           0         
                                                                 
 dense_10 (Dense)            (None, 46, 64)            8256      
                                                      

In [85]:
model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=16, verbose=0)

In [86]:
test_mse_score, test_mae_score = model.evaluate(X_test, test_label)

3/3 [==============================] - 0s 12ms/step - loss: 2.2485 - acc: 0.5797


In [87]:
test_mae_score

0.5797101259231567

In [20]:
#model.fit(X_train,train_label ,epochs=5, batch_size=20)

Epoch 1/5
9/9 [==============================] - 1s 34ms/step - loss: 0.6780 - acc: 0.6056
Epoch 2/5
9/9 [==============================] - 0s 33ms/step - loss: 0.5834 - acc: 0.6500
Epoch 3/5
9/9 [==============================] - 0s 37ms/step - loss: 0.4900 - acc: 0.7000
Epoch 4/5
9/9 [==============================] - 0s 34ms/step - loss: 0.4412 - acc: 0.7889
Epoch 5/5
9/9 [==============================] - 0s 32ms/step - loss: 0.3504 - acc: 0.8944


In [43]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
yhat_probs = model.predict(X_test, verbose=0)
# predict crisp classes for test set
def predict_classes( X, batch_size=20, verbose=1):
    proba = model.predict(X, batch_size=batch_size, verbose=verbose)
    return (proba > 0.5).astype('int32')

print('test_sentiments is:',test_label)
yhat_classes =predict_classes(X_test, verbose=0)
print('yhat_probabilities is:',yhat_probs)
print('predicted classes are:',yhat_classes)
# reduce to 1d array
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]
 
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_label , yhat_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_label , yhat_classes)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_label , yhat_classes)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)

test_sentiments is: [1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0 0 1 1 1 1 0 1 0 1 0 1 0 1 0 1 1 0 0 0 1
 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 1 1 0 0 0 1 0 0 1 1 1 0 0 0 1 0 1]
yhat_probabilities is: [[9.9354196e-01]
 [2.0582676e-03]
 [3.6545610e-01]
 [1.0214841e-01]
 [9.9959052e-01]
 [1.2049201e-01]
 [9.8707509e-01]
 [9.4328221e-05]
 [9.8100376e-01]
 [4.9512088e-03]
 [9.9924767e-01]
 [5.7116657e-02]
 [9.9780017e-01]
 [8.4788013e-01]
 [9.0472364e-01]
 [9.1473138e-01]
 [2.9893517e-03]
 [1.5382171e-03]
 [8.3226025e-01]
 [8.6782467e-01]
 [9.9751467e-01]
 [9.6270263e-01]
 [9.9985242e-01]
 [6.3817644e-01]
 [9.9459231e-01]
 [6.3962042e-01]
 [9.9984860e-01]
 [8.7402904e-01]
 [2.7564764e-03]
 [1.3209879e-03]
 [8.3507555e-05]
 [2.6295781e-03]
 [9.9977362e-01]
 [9.8099494e-01]
 [5.2630365e-02]
 [2.3201498e-01]
 [9.9633002e-01]
 [9.9541551e-01]
 [9.1681522e-01]
 [8.6517730e-06]
 [8.5032660e-01]
 [5.5690098e-06]
 [9.9879944e-01]
 [9.9633002e-01]
 [2.4384997e-06]
 [9.1681522e-01]
 [9.6455061e-01]
 [8.5032660e-01]


In [49]:
predictions = model.predict(X_test)
predictions[9]

array([0.00495121], dtype=float32)